## Load modules

In [1]:
# -*- coding: utf-8 -*-
import os
import numpy as np
import pandas as pd
import networkx as nx
import datetime as dt
import math
import imageio
import geopandas as gpd
import matplotlib.pyplot as plt
import matplotlib.patheffects as pe
from matplotlib.lines import Line2D
from tqdm import tqdm

## Load and prepare mobility data

In [2]:
#load move in data
movein = pd.read_csv('china_data_movein_2023-01-20_2023-01-27.csv')
movein = movein.groupby(["Prov_EN_origin", "Prov_EN_destination", 
                         "Prov_code_origin","Prov_code_destination", "date"], as_index=False).mean()
movein

,Prov_EN_origin,Prov_EN_destination,Prov_code_origin,Prov_code_destination,date,proportion,City_code_origin,City_code_destination
0,Anhui,Anhui,340000.0,340000.0,2023-01-20,3.199600,340921.777778,340973.333333
1,Anhui,Anhui,340000.0,340000.0,2023-01-21,3.271778,340926.222222,340906.666667
2,Anhui,Anhui,340000.0,340000.0,2023-01-22,3.820250,340925.000000,340925.000000
3,Anhui,Anhui,340000.0,340000.0,2023-01-23,4.202889,340921.333333,340980.000000
4,Anhui,Anhui,340000.0,340000.0,2023-01-24,4.214400,340921.333333,340980.000000
...,...,...,...,...,...,...,...,...
7233,Zhejiang,Zhejiang,330000.0,330000.0,2023-01-23,7.494000,330572.500000,330875.000000
7234,Zhejiang,Zhejiang,330000.0,330000.0,2023-01-24,6.438750,330592.500000,330675.000000
7235,Zhejiang,Zhejiang,330000.0,330000.0,2023-01-25,6.553000,330565.000000,330950.000000
7236,Zhejiang,Zhejiang,330000.0,330000.0,2023-01-26,5.674889,330598.888889,330611.111111


## Load and prepare geo dataframe

In [4]:
#the geolocation files are taken from: https://data.humdata.org/dataset/cod-ab-chn?
gdf = gpd.read_file("./chn_adm_ocha_2020_shp/chn_admbnda_adm1_ocha_2020.shp")
gdf = gdf.to_crs("epsg:4087")
gdf['centroid'] = gdf['geometry'].centroid

#gdf = gdf.sort_values(by="ADM1_EN")
#gdf.ADM1_EN.unique()

provs = [n.split(" ")[0] for n in gdf.ADM1_EN.values]
prov = []
for p in provs:
    if p == "Guangxizhuangzu":
        p = "Guangxi"
    if p == "Ningxiahuizu":
        p = "Ningxia"
    if p == "Xinjiangweiwu'erzu":
        p = "Xinjiang"
    if p == "Hong":
        p = "Hongkong"
    if p == "Inner":
        p = "Neimenggu"
    if p == "Tibet":
        p = "Xizang"
    prov.append(p)

gdf['Prov'] = prov

#test that all names in the movein df match the names the geo df
for p in gdf.Prov:
    if p not in movein.Prov_EN_destination.unique():
        print(p)
gdf

,ADM1_EN,ADM1_ZH,ADM1_PCODE,ADM0_EN,ADM0_ZH,ADM0_PCODE,geometry,centroid,Prov
0,Shaanxi Province,陕西省,CN061,China,中国,CN,"POLYGON ((12182196.198 3690899.512, 12182155.6...",POINT (12117529.648 3918802.407),Shaanxi
1,Shanghai Municipality,上海市,CN031,China,中国,CN,"MULTIPOLYGON (((13497188.313 3532249.687, 1349...",POINT (13513059.204 3469228.083),Shanghai
2,Chongqing Municipality,重庆市,CN050,China,中国,CN,"POLYGON ((12194605.319 3531233.106, 12196554.7...",POINT (12006548.598 3344881.461),Chongqing
3,Zhejiang Province,浙江省,CN033,China,中国,CN,"MULTIPOLYGON (((13410441.591 3023558.078, 1341...",POINT (13361045.936 3246664.805),Zhejiang
4,Jiangxi Province,江西省,CN036,China,中国,CN,"POLYGON ((12973045.026 2957731.137, 12975002.1...",POINT (12879597.646 3073324.674),Jiangxi
5,Yunnan Province,云南省,CN053,China,中国,CN,"POLYGON ((11745547.213 2583573.684, 11743538.1...",POINT (11298902.957 2781072.908),Yunnan
6,Shandong Province,山东省,CN037,China,中国,CN,"MULTIPOLYGON (((13389132.178 4007642.574, 1338...",POINT (13145572.369 4042996.759),Shandong
7,Liaoning Province,辽宁省,CN021,China,中国,CN,"MULTIPOLYGON (((13517501.762 4397610.533, 1351...",POINT (13644539.538 4598408.874),Liaoning
8,Tibet Autonomous Region,西藏自治区,CN054,China,中国,CN,"POLYGON ((10032173.736 3136114.959, 10032069.9...",POINT (9817734.212 3526553.493),Xizang
9,Gansu province,甘肃省,CN062,China,中国,CN,"POLYGON ((11856321.078 3809399.199, 11861965.6...",POINT (11235733.740 4210191.292),Gansu


## Create plots

In [5]:
for prov_dest in tqdm(movein.Prov_EN_destination.unique()):
    move_dest = movein[movein.Prov_EN_destination==prov_dest]
    os.mkdir("./results_movein/"+prov_dest)
    
    move_mean = move_dest.groupby("Prov_EN_origin", as_index=False).mean()
    vals = sorted(move_mean.proportion, reverse=True)[:3]
    
    p0 = move_mean[move_mean.proportion== vals[0]]["Prov_EN_origin"].values[0]
    p1 = move_mean[move_mean.proportion== vals[1]]["Prov_EN_origin"].values[0]
    p2 = move_mean[move_mean.proportion== vals[2]]["Prov_EN_origin"].values[0]

    mov0 = move_dest[move_dest.Prov_EN_origin==p0]    
    mov1 = move_dest[move_dest.Prov_EN_origin==p1]
    mov2 = move_dest[move_dest.Prov_EN_origin==p2]
    movs = [mov0, mov1, mov2]
        

    for d in movein.date.unique():
        
        plt.rcParams['font.family'] = "Lucida Console"
        plt.rcParams['font.serif'] = "Lucida Console"
        plt.rcParams['legend.title_fontsize'] = 15
        
        fig = plt.figure(figsize=(20,20), dpi=300)
        fig.subplots_adjust(hspace=100, wspace=0)
        ax = plt.subplot2grid((3, 1), (0, 0), rowspan=1)
        ax_map = plt.subplot2grid((3, 1), (1, 0), rowspan=2)
        
        styles = ['-', '--', ':']
        for m in range(len(movs)):
            mov = movs[m]
            x, y = zip(*sorted(zip(mov.date.values, mov.proportion.values)))
            ax.plot(x,y, label=mov.Prov_EN_origin.values[0], linewidth=5, linestyle=styles[m])
            ax.set_ylabel("Migration Proportion %", fontsize=22)
            ax.set_xlabel("Date", fontsize=22)
            ax.tick_params(axis='both', which='major', labelsize=22)
            ax.legend(fontsize=18)
            ax.axvline(d, linewidth=5, color="k")
            ax.set_title("Highest 3 migration origins over time", fontsize=22)
        
        move = move_dest[move_dest.date==d]  
        
        move.reset_index(inplace=True, drop=True)

        gdf['coords'] = gdf['geometry'].apply(lambda x: x.representative_point().coords[:])
        gdf['coords'] = [coords[0] for coords in gdf['coords']]

        ch_map = gdf.plot(ax=ax_map, figsize=(20,20), color="lightcoral", edgecolor="k")
        for idx, row in gdf.iterrows():
            ch_map.annotate(text=row['Prov'], xy=row['coords'], fontsize=18, color="w", 
                        horizontalalignment='center', path_effects=[pe.withStroke(linewidth=2, foreground="goldenrod")])
            ch_map.set_facecolor("grey")
            ch_map.patch.set_alpha(0.5)
        ch_map.tick_params(axis='both', which='both', bottom=False, top=False, left=False, labelbottom=False,  labelleft=False)
                
        move = move.sort_values(by="proportion", ascending=False)
        move.reset_index(inplace=True, drop=True)
        
        for c in range(len(move)):
            pro = move.Prov_EN_origin.values[c]
            dest_coord = gdf[gdf.Prov==prov_dest].coords.values[0]
            coord = gdf[gdf.Prov==pro].coords.values[0]
            w = move.proportion[c].round(2)
            x = (dest_coord[0], coord[0])
            y = (dest_coord[1], coord[1])
            ch_map.plot(x,y, linewidth=1+w, marker="o", markersize=10, color="slateblue", 
                        label=str(w)+"% migrants from "+pro)
            ch_map.legend(loc="lower right",  title="Highest migration origins", fontsize=12)
        
        plt.tight_layout()
        plt.title("Proportions of Migrants into "+prov_dest+" on "+str(d), size=22)
        
        fig.savefig("./results_movein/"+prov_dest+"/"+prov_dest+"_"+str(d)+"_movein.png", dpi=300)
        plt.close()
       

100%|██████████████████████████████████████████████████████████████████████████████████| 34/34 [13:38<00:00, 24.06s/it]


## Make gifs 

In [6]:
provs = os.listdir("./results_movein/")

for prov_dest in tqdm(provs):
    images = []
    for file in os.listdir("./results_movein/"+prov_dest+"/"):
        images.append(imageio.v2.imread("./results_movein/"+prov_dest+"/"+file))
    
    d1 = movein.date.unique()[0]
    d2 = movein.date.unique()[-1]
    imageio.mimsave("./results_movein/"+prov_dest+"/"+prov_dest+"_"+d1+"_"+d2+".gif", images, duration=2)

100%|██████████████████████████████████████████████████████████████████████████████████| 34/34 [12:08<00:00, 21.44s/it]
